In [18]:
import pandas as pd
import numpy as np

In [65]:
def softmax(x):
    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x
# def softmax(x):

#     y = np.exp(x - np.max(x))
#     f_x = y / np.sum(np.exp(x))
#     return f_x
def normalize(df):
    # return (df-df.min())/(df.max()-df.min())
    return df / np.linalg.norm(df)

In [97]:
ppl_kb = pd.read_csv('/nas/home/gujiashe/trans/yago310_ppls_top10_kb.tsv', sep='\t', index_col=0)
ppl_kb = ppl_kb[ppl_kb["rank"]<=10]
ppl_kb.index = list(range(len(ppl_kb)))
ppl_kb.shape

(31100, 7)

In [135]:
ppl_gpt = pd.read_csv('/nas/home/gujiashe/trans/yago310_ppls_top10_gpt.tsv', sep='\t', index_col=0)
ppl_gpt = ppl_gpt[ppl_gpt["rank"]<=10]
ppl_gpt.index = list(range(len(ppl_gpt)))
ppl_gpt.shape

(31100, 7)

In [139]:
def normalize_score_ppl(ppl):
    ppl_norm = ppl.copy()
    for i in range(len(ppl)//10):
        ppl_norm.loc[i*10: i*10+9, "score"] = normalize(softmax(ppl_norm.loc[i*10: i*10+9, "score"]))
        ppl_norm.loc[i*10: i*10+9, "ppls"] = normalize(ppl_norm.loc[i*10: i*10+9, "ppls"])
    return ppl_norm
ppl_norm = normalize_score_ppl(ppl_gpt)

In [140]:
def multiply_score_ppl(ppl_norm):
    ppl_multiply = ppl_norm.copy()
    for i in range(len(ppl_norm)//10):
        ppl_multiply.loc[i*10: i*10+9, "multiply"] = ppl_multiply.loc[i*10: i*10+9, "score"] * 1/ppl_multiply.loc[i*10: i*10+10, "ppls"]
        ppl_multiply.loc[i*10: i*10+9, "multiply_rank"] = ppl_multiply.loc[i*10: i*10+9, "multiply"].rank()
        ppl_multiply.loc[i*10: i*10+9, "multiply_rank_true"] = ppl_multiply.loc[i*10-1 + ppl_multiply.loc[i*10, 'rank'], "multiply_rank"]  
    return ppl_multiply
ppl_multiply = multiply_score_ppl(ppl_norm)

In [155]:
def plus_score_ppl(ppl_norm):
    ppl_plus = ppl_norm.copy()
    for i in range(len(ppl_norm)//10):
        ppl_plus.loc[i*10: i*10+9, "plus"] = ppl_plus.loc[i*10: i*10+9, "score"] + 1/ppl_plus.loc[i*10: i*10+10, "ppls"]
        ppl_plus.loc[i*10: i*10+9, "plus_rank"] = ppl_plus.loc[i*10: i*10+9, "plus"].rank()
        ppl_plus.loc[i*10: i*10+9, "plus_rank_true"] = ppl_plus.loc[i*10-1 + ppl_plus.loc[i*10, 'rank'], "plus_rank"]  
    return ppl_plus
ppl_plus = plus_score_ppl(ppl_norm)

In [149]:
def print_rank(rank, k):
    print((rank <= k).value_counts())


In [156]:
print_rank(ppl_plus['rank'], 5)
print_rank(ppl_multiply['multiply_rank_true'], 5)
print_rank(ppl_plus['plus_rank_true'], 5)

False    16840
True     14260
Name: rank, dtype: int64
False    17470
True     13630
Name: multiply_rank_true, dtype: int64
False    17380
True     13720
Name: plus_rank_true, dtype: int64
